In [1]:
import os
import json
from tqdm import tqdm

In [2]:
INPUT_FILE = "metadata_full.json"
OUTPUT_DIR = "prompts_batches"
CHUNK_SIZE = 20000
MODEL = "gpt-4"
TEMPERATURE = 0.3

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

with open(INPUT_FILE, "r") as f:
    records = json.load(f)

def build_prompt(record):
    return f"""Ты биоинформатик. Проанализируй метаданные GEO-эксперимента ниже и верни строго JSON со следующими полями:

- GSM_id
- GSE_id
- sc_RNA_seq: True или False — является ли эксперимент single-cell RNA-seq
- taxon: вид животного (например, "Mus musculus", "Rattus norvegicus")
- strain: линия или штамм животного (например, "C57BL/6J", "Wistar", "Sprague-Dawley")
- age_weeks: возраст в неделях (целое или дробное число, если можно извлечь)
- age_bin: одна из категорий возраста: "<4w", "4–8w", "8–20w", ">20w", либо "unknown"
- disease_status: "healthy", "diseased" или "unknown"
- disease_name: если animal diseased — название заболевания (например, "diabetes", "cancer", "asthma"); иначе "none"
- tissue_or_cell_type: тип ткани или клеточной линии (например, "liver", "fibroblast", "bone marrow")


Если в карточке нет точной информации, логично догадывайся из текста. 

=== GEO METADATA START ===
{record["metadata_text"]}
=== GEO METADATA END ===
"""

# === Сохраняем чанки JSONL ===
num_chunks = (len(records) + CHUNK_SIZE - 1) // CHUNK_SIZE

for i in tqdm(range(num_chunks), desc="Сохранение чанков"):
    chunk_records = records[i * CHUNK_SIZE : (i + 1) * CHUNK_SIZE]
    output_path = os.path.join(OUTPUT_DIR, f"batch_chunk_{i+1}.jsonl")
    with open(output_path, "w", encoding="utf-8") as out:
        for record in chunk_records:
            prompt = build_prompt(record)
            jsonl_obj = {
                "custom_id": record["GSM_id"],
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": MODEL,
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": TEMPERATURE,
                    "response_format": "json"
                }
            }
            out.write(json.dumps(jsonl_obj, ensure_ascii=False) + "\n")

print(f"✅ Готово: сохранено {num_chunks} чанков в папке {OUTPUT_DIR}")